# \sigma_{a2}^{Sm} в зависимости от плотности теплоносителя, температуры теплоносителя, температуры топлива и концентрации борной кислоты

## 1. Загрузка данных

In [1]:
import pandas as pd

Загрузка данных

In [2]:
df = pd.read_excel('Input/Data.xlsx')
df.head(3)

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),3S_{tr1}(1),3S_{tr2}(2),\Sigma_{a1}(3),\Sigma_{a2}(4),nuS_{f1}(5),nuS_{f2}(6),\Sigma_{f1}(7),\Sigma_{f2}(8),\Sigma_{1->2}(9),k_{inf}(10),\sigma_{a2}^{Xe},\sigma_{a2}^{Sm}
0,0.7235,575.0,1027.0,600.0,0.661853,2.568298,0.009376,0.085584,0.006677,0.133250,0.002620,0.054750,0.017392,1.261041,1072970.0,35421.398
1,0.7750,448.0,1043.0,1500.0,0.679602,2.874687,0.009581,0.098339,0.006723,0.139256,0.002636,0.057218,0.018726,1.174265,1141000.0,36147.801
2,0.9125,370.5,1418.0,750.0,0.727336,3.668742,0.009908,0.102815,0.006844,0.153823,0.002681,0.063203,0.022463,1.249619,1308480.0,39214.000


In [3]:
names = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)',
 '\sigma_{a2}^{Sm}']

In [4]:
df = df[names]
df.head(3)

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),\sigma_{a2}^{Sm}
0,0.7235,575.0,1027.0,600.0,35421.398
1,0.7750,448.0,1043.0,1500.0,36147.801
2,0.9125,370.5,1418.0,750.0,39214.000


### 2 Нормализация признаков

In [5]:
features = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)']

In [6]:
df_features_norm = df.copy()

In [7]:
for i in range(len(features)):
    df_features_norm[features[i]] = (df[features[i]]-df[features[i]].mean())/df[features[i]].std()

In [8]:
df_features_norm

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),\sigma_{a2}^{Sm}
0,-0.316426,1.416398,-0.039094,-1.035019,35421.398
1,0.007922,-0.005506,-0.001946,0.006984,36147.801
2,0.873902,-0.873204,0.868693,-0.861352,39214.000
3,-0.858058,0.862191,-0.872586,0.875321,32457.400
4,-0.425067,-0.439355,0.433373,-1.295520,36604.801
...,...,...,...,...,...
252,-0.844527,-0.886762,-1.539169,-0.603564,34268.102
253,0.887433,0.848634,0.202111,1.133108,36142.898
254,0.021453,-0.019064,1.072750,-1.471900,38047.500
255,-1.710507,1.716332,-0.668529,0.264771,29368.699


In [9]:
df_features_norm.sample(frac=1) 

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),\sigma_{a2}^{Sm}
113,0.251479,1.214694,0.732655,0.685373,34998.398
114,1.117459,-1.388399,1.603294,-0.182963,38248.898
50,1.144521,-0.059738,0.052470,0.821050,37526.500
38,0.603283,-0.385123,1.684916,1.580844,35624.102
179,-0.546846,0.550362,1.045542,1.051702,33282.102
...,...,...,...,...,...
154,1.455734,-0.913877,0.011657,-0.305074,39460.699
172,-0.438598,-1.293496,0.501392,1.594412,32927.398
218,1.482796,-0.507144,1.562483,0.536127,38552.898
133,1.347485,0.713056,1.426445,-0.630700,39383.000


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_features_norm[['\\rho(g/cm^3)', 'T_c(K)', 'T_f(K)', 'c_b(ppm)']],
                                                    df_features_norm[['\sigma_{a2}^{Sm}']],
                                                    test_size=0.33,
                                                    random_state=42)

# Нейросетевые модели

In [11]:
%%time

# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf

# Function to create model, required for KerasRegressor
def create_model(activation='relu', learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation=activation))
    model.add(Dense(1, kernel_initializer='normal'))
    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = learn_rate, rho = 0.99, momentum = momentum, epsilon=1e-10)
    
    # Compile model
    model.compile(loss='mse', optimizer=optimizer)
    return model

X = X_train
Y = y_train

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 15, 20]
epochs = [700, 1000, 1300]
activation = ['tanh', 'sigmoid', 'linear']
learn_rate = [0.01, 0.05, 0.1, 0.3, 0.8]
momentum = [0]

param_grid = dict(batch_size=batch_size,
                  epochs=epochs,
                  activation=activation,
                  learn_rate=learn_rate,
                  momentum=momentum
                 )
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=5)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -246422.281250 using {'activation': 'linear', 'batch_size': 10, 'epochs': 1300, 'learn_rate': 0.05, 'momentum': 0}
-1237628160.000000 (20867887.567456) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.01, 'momentum': 0}
-1102455398.400000 (19670516.714980) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.05, 'momentum': 0}
-944720729.600000 (18167023.949020) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.1, 'momentum': 0}
-438771456.000000 (12221343.838147) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.3, 'momentum': 0}
-7907770.900000 (643280.312571) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 700, 'learn_rate': 0.8, 'momentum': 0}
-1222955161.600000 (20741928.554395) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 0.01, 'momentum': 0}
-1034151462.400000 (19031159.982042) with: {'activation': 'tanh', 'batch_size': 10, 

In [12]:
Best: -246422.281250 using {'activation': 'linear', 'batch_size': 10, 'epochs': 1300, 'learn_rate': 0.05, 'momentum': 0}

SyntaxError: invalid syntax (<ipython-input-12-9b588eecbaae>, line 1)

## Модель 1

In [55]:
def create_model():

    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation='linear'))
    model.add(Dense(1, kernel_initializer='normal'))

    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.05, rho = 0.99, momentum = 0, epsilon=1e-10)
    
    # Compile model
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [56]:
model = create_model()
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [57]:
# Обучение модели

EPOCHS = 1300
batch_size = 10

history = model.fit(
  X, Y,
  epochs=EPOCHS, batch_size = batch_size,verbose=0)

In [58]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(10)

,loss,epoch
1290,244616.343750,1290
1291,245973.343750,1291
1292,246888.046875,1292
1293,250267.140625,1293
1294,255769.843750,1294
1295,253500.671875,1295
1296,249783.859375,1296
1297,250728.562500,1297
1298,247559.718750,1298
1299,250136.093750,1299


In [59]:
import statsmodels as statsmodels
import statsmodels.api as sm

RMSPE = statsmodels.tools.eval_measures.rmspe(y_test.values, model.predict(X_test))
RMSPE

array([0.1735863])

In [60]:
import statsmodels as statsmodels
import statsmodels.api as sm

RMSPE = statsmodels.tools.eval_measures.rmspe(y_train.values, model.predict(X_train))
RMSPE

array([0.14052459])